**Movie attributes**

https://developers.themoviedb.org/3/movies/get-movie-details
1. box office gross revenue (Int'l + US) (API: movie details - 'revenue')
    - international gross
    - US gross
2. Movie name(API: movie details - 'original_title')
3. Budget (API: movie details - 'budget') 
4. IMDB number (or any other unique movie ID) (API: movie details - 'id' or 'imdb_id)
5. Release Date (year) (API: movie details - 'release_date') 
. Popularity (API: movie details - 'popularity') 
6. Number of theaters (or how widespread release)
7. Country of Release (US only)
8. Genre(s)
9. Actors/Actresses
    - Separate table (Join table)
10. Reviews (time permitting)
11. Awards (time permitting)

**Join Table**
Use /movie/{movie_id}/credits API
https://developers.themoviedb.org/3/movies/get-movie-credits
1. Movie ID
2. Actor/Actress name or ID

**Movie Actors/Actresses**
1. Name 
2. Ethnicity - web scrape https://ethnicelebs.com/
3. A-list or not (how to define this?)
4. Awards 

## API call to obtain list of most popular movies by Release Year

In [184]:
def get_movies_by_year(year, page_num):
    import config
    import requests
    import json
    
    api_url = 'https://api.themoviedb.org/3/discover/movie'
    api_filter = '&primary_release_year='+year+'&sort_by=popularity.desc&page='
    api_call = api_url + config.api_key + api_filter
    response = requests.get(api_call+str(page_num))
    if response.status_code == 200:
        result = response.json() 
        movie_ids = []
        for movie in result['results']:
            movie_ids.append(movie['id'])
        return movie_ids
    else:
        print("There was an error:", response.status_code)

In [82]:
def api_get_movie_details(movie):
    api_url = 'https://api.themoviedb.org/3/movie/'
    api_call = api_url + str(movie) + config.api_key
    response = requests.get(api_call)
    return response

In [31]:
def create_movie_detail_dict(result):
    movie_dict = {}
    movie_dict['id'] = result['id']
    movie_dict['imdb_id'] = result['imdb_id']
    movie_dict['name'] = result['title']
    movie_dict['budget'] = result['budget']
    movie_dict['revenue'] = result['revenue']
    movie_dict['popularity'] = result['popularity']
    movie_dict['release_date'] = result['release_date']
    movie_dict['vote_average'] = result['vote_average']
    movie_dict['vote_count'] = result['vote_count']
    movie_dict['runtime'] = result['runtime']
    for country in result['production_countries']:
        if country['name'] == 'United States of America':
            movie_dict['production_country'] = 'US'
    if 'production_country' not in movie_dict:
        movie_dict['production_country'] = 'non-US'
    movie_dict['language'] = result['original_language']
    return movie_dict

In [61]:
def create_movie_genre_dict(result):
    genres_list = []
    genre_dict = {}
    for genre in result['genres']:
        genres_list.append(genre['name'])
    genre_dict[result['id']] = genres_list
    return genre_dict

In [46]:
def movie_formatting(movie):
    movie_tuple = (movie['id'], movie['imdb_id'], movie['name'], movie['budget'], 
    movie['revenue'], movie['popularity'], 
    movie['release_date'], movie['vote_average'], 
    movie['vote_count'], movie['runtime'], movie['production_country'], movie['language'])
    return movie_tuple

In [176]:
def add_to_db_movie(movie_tuple):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO details (id,
       imdb_id,
       name,
       budget,
       revenue,
       popularity,
       release_date,
       vote_average,
       vote_count,
       runtime,
       production_country,
       language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE id = id;"""
    c.execute(stmt, movie_tuple)
    cnx.commit()
    c.close()
    cnx.close()

In [180]:
def add_to_db_movies(movie_list):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO details (id,
       imdb_id,
       name,
       budget,
       revenue,
       popularity,
       release_date,
       vote_average,
       vote_count,
       runtime,
       production_country,
       language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    c.executemany(stmt, movie_list)
    cnx.commit()
    c.close()
    cnx.close()

In [99]:
genre = {385922: []}

In [107]:
a = genre.items()

In [167]:
def genre_formatting(genre):
    for key, value in genre.items():
        if value:
            many_genres = []
            for genre in value:
                genre_tuple = (key, genre)
                many_genres.append(genre_tuple)
            return many_genres
        else:
            return (key, '')

In [182]:
def add_to_db_genres(genre_list):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO genre (id, genre)
       VALUES (%s, %s) ON DUPLICATE KEY UPDATE id = id;"""
    c.executemany(stmt, genre_list)
    cnx.commit()
    c.close()
    cnx.close()

In [168]:
def get_and_format_movies(year, page_num):
    movie_ids = get_movies_by_year(year, page_num)
    for movie in movie_ids:
        response = api_get_movie_details(movie)
        if response.status_code == 200:
            result = response.json()
            movie_dict = create_movie_detail_dict(result)
            movie_tuple = movie_formatting(movie_dict)
            print(movie_dict)
            add_to_db_movie(movie_tuple)
        
            genre_dict = create_movie_genre_dict(result)
            print(genre_dict)
            genre_list = genre_formatting(genre_dict)
            add_to_db_genres(genre_list)

In [183]:
years = ['2017']
for year in years:
    for page_num in range(6, 20):
        get_and_format_movies(year, page_num)

{'id': 324852, 'imdb_id': 'tt3469046', 'name': 'Despicable Me 3', 'budget': 80000000, 'revenue': 1031552585, 'popularity': 12.43, 'release_date': '2017-06-15', 'vote_average': 6.3, 'vote_count': 4301, 'runtime': 96, 'production_country': 'US', 'language': 'en'}
{324852: ['Action', 'Animation', 'Comedy', 'Family', 'Adventure']}
{'id': 289222, 'imdb_id': 'tt1730768', 'name': "The Zookeeper's Wife", 'budget': 20000000, 'revenue': 23299081, 'popularity': 9.344, 'release_date': '2017-03-31', 'vote_average': 7.4, 'vote_count': 774, 'runtime': 126, 'production_country': 'US', 'language': 'en'}
{289222: ['Drama', 'History']}
{'id': 372343, 'imdb_id': 'tt1758810', 'name': 'The Snowman', 'budget': 35000000, 'revenue': 43203413, 'popularity': 14.988, 'release_date': '2017-10-12', 'vote_average': 5.1, 'vote_count': 1287, 'runtime': 119, 'production_country': 'US', 'language': 'en'}
{372343: ['Crime', 'Thriller']}
{'id': 407445, 'imdb_id': 'tt5716464', 'name': 'Breathe', 'budget': 0, 'revenue': 477

{448358: ['Thriller', 'Drama']}
{'id': 440597, 'imdb_id': 'tt5322012', 'name': 'Wish Upon', 'budget': 12000000, 'revenue': 14301505, 'popularity': 10.879, 'release_date': '2017-07-07', 'vote_average': 5.4, 'vote_count': 738, 'runtime': 90, 'production_country': 'non-US', 'language': 'en'}
{440597: ['Thriller', 'Horror', 'Fantasy']}
{'id': 424488, 'imdb_id': 'tt4899370', 'name': 'Megan Leavey', 'budget': 0, 'revenue': 13406883, 'popularity': 12.574, 'release_date': '2017-06-09', 'vote_average': 7.1, 'vote_count': 401, 'runtime': 116, 'production_country': 'US', 'language': 'en'}
{424488: ['Drama', 'War']}
{'id': 437670, 'imdb_id': 'tt6471264', 'name': 'Suck Me Shakespeer 3', 'budget': 0, 'revenue': 0, 'popularity': 13.621, 'release_date': '2017-10-26', 'vote_average': 6.1, 'vote_count': 183, 'runtime': 119, 'production_country': 'non-US', 'language': 'de'}
{437670: ['Comedy']}
{'id': 438740, 'imdb_id': 'tt6537238', 'name': 'Salyut-7', 'budget': 15000000, 'revenue': 13767037, 'popularity

{'id': 392982, 'imdb_id': 'tt5301662', 'name': 'Marshall', 'budget': 12000000, 'revenue': 10051659, 'popularity': 13.532, 'release_date': '2017-10-13', 'vote_average': 7.3, 'vote_count': 282, 'runtime': 118, 'production_country': 'US', 'language': 'en'}
{392982: ['Drama']}
{'id': 411741, 'imdb_id': 'tt5962210', 'name': 'Ingrid Goes West', 'budget': 0, 'revenue': 24931915, 'popularity': 11.278, 'release_date': '2017-08-11', 'vote_average': 6.5, 'vote_count': 449, 'runtime': 97, 'production_country': 'US', 'language': 'en'}
{411741: ['Drama', 'Comedy']}
{'id': 451644, 'imdb_id': 'tt5610626', 'name': 'Dragonheart: Battle for the Heartfire', 'budget': 0, 'revenue': 0, 'popularity': 7.246, 'release_date': '2017-07-09', 'vote_average': 5.9, 'vote_count': 99, 'runtime': 98, 'production_country': 'US', 'language': 'en'}
{451644: ['Adventure']}
{'id': 423646, 'imdb_id': 'tt5723272', 'name': 'In the Fade', 'budget': 0, 'revenue': 321011, 'popularity': 9.03, 'release_date': '2017-11-23', 'vote_av

{353569: ['Comedy', 'Drama']}
{'id': 416477, 'imdb_id': 'tt5462602', 'name': 'The Big Sick', 'budget': 5000000, 'revenue': 56303596, 'popularity': 13.132, 'release_date': '2017-03-30', 'vote_average': 7.5, 'vote_count': 1365, 'runtime': 120, 'production_country': 'US', 'language': 'en'}
{416477: ['Comedy', 'Drama', 'Romance']}
{'id': 460846, 'imdb_id': 'tt3501112', 'name': 'Security', 'budget': 15000000, 'revenue': 0, 'popularity': 11.626, 'release_date': '2017-03-04', 'vote_average': 6.0, 'vote_count': 366, 'runtime': 87, 'production_country': 'US', 'language': 'en'}
{460846: ['Action', 'Thriller', 'Crime']}
{'id': 455839, 'imdb_id': 'tt4815122', 'name': 'Time Trap', 'budget': 3, 'revenue': 0, 'popularity': 10.711, 'release_date': '2017-05-19', 'vote_average': 6.3, 'vote_count': 86, 'runtime': 95, 'production_country': 'US', 'language': 'en'}
{455839: ['Adventure', 'Science Fiction']}
{'id': 395993, 'imdb_id': 'tt3881784', 'name': 'Stronger', 'budget': 30000000, 'revenue': 5796490, 'p

{429467: ['Drama', 'Horror', 'Romance', 'Thriller']}
{'id': 445030, 'imdb_id': 'tt5914996', 'name': 'No Game No Life: Zero', 'budget': 0, 'revenue': 6000000, 'popularity': 8.741, 'release_date': '2017-07-15', 'vote_average': 7.4, 'vote_count': 98, 'runtime': 106, 'production_country': 'non-US', 'language': 'ja'}
{445030: ['Fantasy', 'Comedy', 'Animation']}
{'id': 430447, 'imdb_id': 'tt6336356', 'name': "Mary and the Witch's Flower", 'budget': 0, 'revenue': 0, 'popularity': 12.046, 'release_date': '2017-07-08', 'vote_average': 7.1, 'vote_count': 361, 'runtime': 103, 'production_country': 'non-US', 'language': 'ja'}
{430447: ['Fantasy', 'Adventure', 'Animation']}
{'id': 387426, 'imdb_id': 'tt3967856', 'name': 'Okja', 'budget': 50000000, 'revenue': 0, 'popularity': 13.494, 'release_date': '2017-06-28', 'vote_average': 7.5, 'vote_count': 2270, 'runtime': 122, 'production_country': 'US', 'language': 'en'}
{387426: ['Adventure', 'Drama', 'Science Fiction', 'Action']}
{'id': 180863, 'imdb_id'

{365942: ['Romance', 'Adventure', 'Science Fiction', 'Drama']}
{'id': 345317, 'imdb_id': 'tt4494718', 'name': 'Final Portrait', 'budget': 0, 'revenue': 1725841, 'popularity': 7.32, 'release_date': '2017-08-03', 'vote_average': 6.0, 'vote_count': 78, 'runtime': 87, 'production_country': 'US', 'language': 'en'}
{345317: ['Drama', 'Comedy']}
{'id': 354556, 'imdb_id': 'tt4600952', 'name': 'Guardians', 'budget': 5400000, 'revenue': 4803565, 'popularity': 9.717, 'release_date': '2017-02-14', 'vote_average': 4.5, 'vote_count': 381, 'runtime': 100, 'production_country': 'non-US', 'language': 'ru'}
{354556: ['Action', 'Fantasy', 'Science Fiction']}
{'id': 340027, 'imdb_id': 'tt3704700', 'name': 'Brain on Fire', 'budget': 0, 'revenue': 0, 'popularity': 12.223, 'release_date': '2017-02-22', 'vote_average': 7.2, 'vote_count': 739, 'runtime': 88, 'production_country': 'non-US', 'language': 'en'}
{340027: ['Drama']}
{'id': 467936, 'imdb_id': 'tt6040662', 'name': 'The Children Act', 'budget': 0, 'rev

{376134: ['Drama', 'Romance', 'Science Fiction', 'Thriller', 'Mystery']}
{'id': 419709, 'imdb_id': 'tt6018306', 'name': 'Last Flag Flying', 'budget': 0, 'revenue': 0, 'popularity': 6.901, 'release_date': '2017-11-03', 'vote_average': 6.9, 'vote_count': 217, 'runtime': 125, 'production_country': 'US', 'language': 'en'}
{419709: ['Drama', 'Comedy']}
{'id': 453278, 'imdb_id': 'tt6217608', 'name': 'The Rider', 'budget': 0, 'revenue': 0, 'popularity': 10.097, 'release_date': '2017-08-16', 'vote_average': 7.4, 'vote_count': 166, 'runtime': 105, 'production_country': 'US', 'language': 'en'}
{453278: ['Drama', 'Western']}
{'id': 420245, 'imdb_id': 'tt3622110', 'name': "Don't Knock Twice", 'budget': 0, 'revenue': 0, 'popularity': 9.141, 'release_date': '2017-02-03', 'vote_average': 5.0, 'vote_count': 367, 'runtime': 93, 'production_country': 'non-US', 'language': 'en'}
{420245: ['Horror']}
{'id': 423453, 'imdb_id': 'tt4466894', 'name': 'Sahara', 'budget': 0, 'revenue': 7800000, 'popularity': 12

{397567: ['Drama', 'Fantasy', 'Action']}
{'id': 407449, 'imdb_id': 'tt5859238', 'name': 'Lucky', 'budget': 0, 'revenue': 955925, 'popularity': 10.349, 'release_date': '2017-09-29', 'vote_average': 7.4, 'vote_count': 230, 'runtime': 88, 'production_country': 'US', 'language': 'en'}
{407449: ['Drama']}
{'id': 470114, 'imdb_id': 'tt5442456', 'name': '24 Hours to Live', 'budget': 0, 'revenue': 0, 'popularity': 10.847, 'release_date': '2017-10-26', 'vote_average': 5.6, 'vote_count': 314, 'runtime': 94, 'production_country': 'US', 'language': 'en'}
{470114: ['Action', 'Science Fiction', 'Thriller', 'Mystery']}
{'id': 436274, 'imdb_id': 'tt6097798', 'name': 'Radius', 'budget': 0, 'revenue': 0, 'popularity': 7.294, 'release_date': '2017-08-25', 'vote_average': 6.3, 'vote_count': 192, 'runtime': 93, 'production_country': 'non-US', 'language': 'en'}
{436274: ['Fantasy', 'Science Fiction', 'Thriller']}
{'id': 400136, 'imdb_id': 'tt5776208', 'name': 'Resident Evil: Vendetta', 'budget': 0, 'revenue

{437425: ['Romance', 'Comedy']}
{'id': 415401, 'imdb_id': 'tt5814592', 'name': 'The Party', 'budget': 0, 'revenue': 3300000, 'popularity': 9.023, 'release_date': '2017-09-13', 'vote_average': 6.4, 'vote_count': 244, 'runtime': 71, 'production_country': 'non-US', 'language': 'en'}
{415401: ['Comedy', 'Drama']}
{'id': 437739, 'imdb_id': 'tt7029854', 'name': 'The Mansion', 'budget': 0, 'revenue': 0, 'popularity': 6.52, 'release_date': '2017-06-21', 'vote_average': 5.1, 'vote_count': 242, 'runtime': 100, 'production_country': 'non-US', 'language': 'fr'}
{437739: ['Horror', 'Comedy']}
{'id': 346671, 'imdb_id': 'tt2937366', 'name': 'Little Evil', 'budget': 0, 'revenue': 0, 'popularity': 9.515, 'release_date': '2017-08-08', 'vote_average': 5.7, 'vote_count': 560, 'runtime': 94, 'production_country': 'US', 'language': 'en'}
{346671: ['Comedy', 'Horror']}
{'id': 351460, 'imdb_id': 'tt1241317', 'name': 'Death Note', 'budget': 40000000, 'revenue': 0, 'popularity': 8.015, 'release_date': '2017-08-

{452068: ['Thriller']}
{'id': 466876, 'imdb_id': 'tt5851786', 'name': 'Gnome Alone', 'budget': 0, 'revenue': 0, 'popularity': 9.271, 'release_date': '2017-11-02', 'vote_average': 6.1, 'vote_count': 85, 'runtime': 85, 'production_country': 'US', 'language': 'en'}
{466876: ['Adventure', 'Family', 'Animation', 'Comedy', 'Science Fiction', 'Fantasy']}
{'id': 417830, 'imdb_id': 'tt6003368', 'name': 'Diary of a Wimpy Kid: The Long Haul', 'budget': 22000000, 'revenue': 40120144, 'popularity': 10.216, 'release_date': '2017-05-19', 'vote_average': 5.7, 'vote_count': 210, 'runtime': 91, 'production_country': 'US', 'language': 'en'}
{417830: ['Comedy', 'Family']}
{'id': 459928, 'imdb_id': 'tt5143226', 'name': '12 Feet Deep', 'budget': 0, 'revenue': 0, 'popularity': 6.966, 'release_date': '2017-06-20', 'vote_average': 5.3, 'vote_count': 209, 'runtime': 85, 'production_country': 'US', 'language': 'en'}
{459928: ['Thriller', 'Horror']}
{'id': 463906, 'imdb_id': 'tt2569088', 'name': 'The Saint', 'bud

In [ ]:
years = ['2009', '2008']
years completed = 2016, 2018(16-20), 2016(16-20), 2015, 2014, 2013, 2012, 2011, 2010

In [148]:
def get_and_format_actors():
    movie_ids = get_movie_list()
    actor_tuples = []
    for movie in movie_ids:
        actor_dict = get_actors_api_call(movie[0])
        actor_movie_list = actor_movie_formatting(actor_dict)
        actor_tuples.append(actor_dict)
        add_to_db_actor_movie(actor_movie_list)
    return actor_tuples

In [151]:
get_and_format_actors()

[[],
 [{'movie_id': 96936, 'actor_id': 1053170, 'order': 0, 'name': 'Katie Chang'},
  {'movie_id': 96936,
   'actor_id': 969140,
   'order': 1,
   'name': 'Israel Broussard'},
  {'movie_id': 96936, 'actor_id': 10990, 'order': 2, 'name': 'Emma Watson'},
  {'movie_id': 96936,
   'actor_id': 1172491,
   'order': 3,
   'name': 'Claire Julien'},
  {'movie_id': 96936,
   'actor_id': 527313,
   'order': 4,
   'name': 'Taissa Farmiga'}],
 [{'movie_id': 97020, 'actor_id': 92404, 'order': 0, 'name': 'Joel Kinnaman'},
  {'movie_id': 97020, 'actor_id': 64, 'order': 1, 'name': 'Gary Oldman'},
  {'movie_id': 97020, 'actor_id': 2232, 'order': 2, 'name': 'Michael Keaton'},
  {'movie_id': 97020, 'actor_id': 37260, 'order': 3, 'name': 'Abbie Cornish'},
  {'movie_id': 97020,
   'actor_id': 17183,
   'order': 4,
   'name': 'Jackie Earle Haley'}],
 [{'movie_id': 97051, 'actor_id': 29221, 'order': 0, 'name': 'Brittany Snow'},
  {'movie_id': 97051, 'actor_id': 27993, 'order': 1, 'name': 'Jeffrey Combs'},
  {

In [135]:
def get_actors_api_call(movie):
    api_url = 'https://api.themoviedb.org/3/movie/'
    api_call = api_url + str(movie) + '/credits' + config.api_key
    response = requests.get(api_call)
    if response.status_code == 200:
        result = response.json()
        actor_results = []
        for person in result['cast'][:5]:
            actor_dict = {}
            actor_dict['movie_id']  = movie
            actor_dict['actor_id'] = person['id']
            actor_dict['order'] = person['order']
            actor_dict['name'] =  person['name']
            actor_results.append(actor_dict)

In [145]:
get_actors_api_call(189)

[{'movie_id': 189, 'actor_id': 2295, 'order': 0, 'name': 'Mickey Rourke'},
 {'movie_id': 189, 'actor_id': 56731, 'order': 1, 'name': 'Jessica Alba'},
 {'movie_id': 189, 'actor_id': 16851, 'order': 2, 'name': 'Josh Brolin'},
 {'movie_id': 189,
  'actor_id': 24045,
  'order': 3,
  'name': 'Joseph Gordon-Levitt'},
 {'movie_id': 189, 'actor_id': 5916, 'order': 4, 'name': 'Rosario Dawson'}]

In [150]:
def get_movie_list():
    import pandas as pd
    import mysql.connector
    import config
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()
#     q = """SELECT id FROM details;"""
    
    q =  """SELECT movies.details.id FROM movies.movie_actors
RIGHT JOIN movies.details
ON movies.details.id = movies.movie_actors.id
WHERE ISNULL(actor);"""
    c.execute(q)
    movie_ids = c.fetchall()
    return movie_ids

In [92]:
def actor_movie_formatting(actor_dict):
    formatted_actor_movies = []
    for actor_movie in actor_dict:
        actor_movie_tuple = (actor_movie['movie_id'], actor_movie['name'])
        formatted_actor_movies.append(actor_movie_tuple)
    return formatted_actor_movies

In [93]:
def add_to_db_actor_movie(actor_movie_list):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO movie_actors (id, actor) VALUES (%s, %s)"""
    c.executemany(stmt, actor_movie_list)
    cnx.commit()
    c.close()
    cnx.close()

In [43]:
# Original Code to pull movie details
api_url = 'https://api.themoviedb.org/3/movie/'
movie_details = []
for movie in movie_ids[:2]:
    api_call = api_url + str(movie) + config.api_key
    print(api_call)
    response = requests.get(api_call)
    result = response.json()
    movie_details.append(result)

NameError: name 'movie_ids' is not defined

In [ ]:
movies_missing_details = []
movie_details = []
movie_genres = []

for movie in movie_ids:
    movie_dict = {}
    genres_dict = {}
    
    api_call = api_url + str(movie) + config.api_key
    response = requests.get(api_call)
    if response.status_code != 200:
        movies_missing_details.append(movie)
    else:
        result = response.json()
        movie_dict['id'] = result['id']
        movie_dict['imdb_id'] = result['imdb_id']
        movie_dict['name'] = result['title']
#         print(result['title'])
        movie_dict['budget'] = result['budget']
        movie_dict['revenue'] = result['revenue']
        movie_dict['popularity'] = result['popularity']
        movie_dict['release_date'] = result['release_date']
        movie_dict['vote_average'] = result['vote_average']
        movie_dict['vote_count'] = result['vote_count']
        movie_dict['runtime'] = result['runtime']
        for country in result['production_countries']:
#             print(country)
            if country['name'] == 'United States of America':
#                 print(country['name'])
                movie_dict['production_country'] = 'US'
        if 'production_country' not in movie_dict:
            movie_dict['production_country'] = 'non-US'
        movie_dict['language'] = result['original_language']
        
        genres_list = []
        for genre in result['genres']:
            genres_list.append(genre['name'])
        genres_dict[movie] = genres_list
        movie_genres.append(genres_dict)
        movie_details.append(movie_dict)
        

In [44]:
import pandas as pd
import mysql.connector
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
c = cnx.cursor()

In [55]:
q = """CREATE TABLE details (
       id int(6) PRIMARY KEY,
       imdb_id TEXT,
       name TEXT,
       budget REAL,
       revenue REAL,
       popularity REAL,
       release_date DATE,
       vote_average REAL,
       vote_count INTEGER,
       runtime REAL,
       production_country TEXT,
       language TEXT
     ) ENGINE=InnoDB"""

In [56]:
c = cnx.cursor()
c.execute(q)
cnx.commit()

ProgrammingError: 1050 (42S01): Table 'details' already exists

In [57]:
def movie_formatting(details):
    formatted_movies = []
    for movie in details:
        movie_tuple = (movie['id'], movie['imdb_id'], movie['name'], movie['budget'], 
        movie['revenue'], movie['popularity'], 
        movie['release_date'], movie['vote_average'], movie['vote_count'], movie['runtime'], movie['production_country'], movie['language'])
        formatted_movies.append(movie_tuple)
    return formatted_movies

In [58]:
def add_to_db_movies(movie_list):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO details (id,
       imdb_id,
       name,
       budget,
       revenue,
       popularity,
       release_date,
       vote_average,
       vote_count,
       runtime,
       production_country,
       language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    c.executemany(stmt, movie_list)
    cnx.commit()
    c.close()
    cnx.close()

In [59]:
movie_formatting(movie_details)

[(429203,
  'tt2837574',
  'The Old Man & the Gun',
  0,
  11277120,
  261.026,
  '2018-09-27',
  6.4,
  515,
  93,
  'US',
  'en'),
 (299536,
  'tt4154756',
  'Avengers: Infinity War',
  300000000,
  2046239637,
  67.449,
  '2018-04-25',
  8.3,
  15210,
  149,
  'US',
  'en'),
 (324857,
  'tt4633694',
  'Spider-Man: Into the Spider-Verse',
  90000000,
  375450417,
  49.565,
  '2018-12-06',
  8.4,
  5132,
  117,
  'US',
  'en'),
 (363088,
  'tt5095030',
  'Ant-Man and the Wasp',
  140000000,
  622674139,
  40.387,
  '2018-07-04',
  7.0,
  6988,
  119,
  'US',
  'en'),
 (404368,
  'tt5848272',
  'Ralph Breaks the Internet',
  175000000,
  529221154,
  36.15,
  '2018-11-20',
  7.2,
  3206,
  112,
  'US',
  'en'),
 (284054,
  'tt1825683',
  'Black Panther',
  200000000,
  1346739107,
  39.031,
  '2018-02-13',
  7.4,
  13224,
  134,
  'US',
  'en'),
 (383498,
  'tt5463162',
  'Deadpool 2',
  110000000,
  741547413,
  39.787,
  '2018-05-10',
  7.5,
  9509,
  121,
  'US',
  'en'),
 (424783,


In [84]:
# add_to_db_movies(movie_formatting(movie_details))

In [74]:
import pandas as pd
import mysql.connector
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
c = cnx.cursor()

In [77]:
q = """CREATE TABLE genre (
       id int(6),
       genre VARCHAR(255),
       PRIMARY KEY (id, genre)
     ) ENGINE=InnoDB"""

In [75]:
q = """ DROP TABLE genre;"""

In [78]:
c.execute(q)
cnx.commit()

In [79]:
def genre_formatting(details):
    formatted_genres = []
    for genre in details:
        for key, value in genre.items():
            for genre in value:
                genre_tuple = ( key ,genre )
                formatted_genres.append(genre_tuple)
    return formatted_genres

In [81]:
def add_to_db_genres(genres_list):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO genre (id, genre)
       VALUES (%s, %s)"""
    c.executemany(stmt, genres_list)
    cnx.commit()
    c.close()
    cnx.close()

In [82]:
add_to_db_genres(genre_formatting(movie_genres))

In [106]:
movie_ids = []

for movie in result['results']:
    movie_ids.append(movie['id'])

In [107]:
movie_ids

[429203,
 299536,
 324857,
 363088,
 404368,
 284054,
 383498,
 424783,
 335983,
 440471,
 338952,
 428078,
 424694,
 297802,
 351286,
 260513,
 399402,
 332562,
 353081,
 375588]

In [ ]:
# Original Code to pull movie details
movie_details = []
for movie in movie_ids[:2]:
    api_call = api_url + str(movie) + config.api_key
    print(api_call)
    response = requests.get(api_call)
    result = response.json()
    movie_details.append(result)

In [108]:
movies_missing_cast = []
api_url = 'https://api.themoviedb.org/3/movie/'
api_call = api_url + str(movie_ids[0]) + '/credits' + config.api_key
response = requests.get(api_call)
if response.status_code != 200:
    movies_missing_cast.append(movie)
else:
    result = response.json()

In [109]:
api_call

'https://api.themoviedb.org/3/movie/429203/credits?api_key=09788a07dd795fd26ca2bc24e9a61cb0'

In [111]:
result.keys()

dict_keys(['id', 'cast', 'crew'])

In [112]:
result['id']

429203

In [117]:
result['cast'][3]

{'cast_id': 4,
 'character': 'Teddy',
 'credit_id': '58d4254b9251411f900193cb',
 'gender': 2,
 'id': 2047,
 'name': 'Danny Glover',
 'order': 3,
 'profile_path': '/jSNTEnm0Sxm8FRtoBfJmhmQyozH.jpg'}

In [130]:
movies_missing_cast = []
movie_actors = []

for movie in movie_ids:

    api_url = 'https://api.themoviedb.org/3/movie/'
    api_call = api_url + str(movie) + '/credits' + config.api_key
    response = requests.get(api_call)
    if response.status_code != 200:
        movies_missing_cast.append(movie)
    else:
        result = response.json()
        for person in result['cast']:
            actor_dict = {}
            actor_dict['movie_id']  = movie
            actor_dict['actor_id'] = person['id']
            actor_dict['order'] = person['order']
            actor_dict['name'] =  person['name']
            movie_actors.append(actor_dict)
        

In [129]:
movie_actors

[{'movie_id': 429203, 'actor_id': 4135, 'order': 0, 'name': 'Robert Redford'},
 {'movie_id': 429203, 'actor_id': 5606, 'order': 1, 'name': 'Sissy Spacek'},
 {'movie_id': 429203, 'actor_id': 1893, 'order': 2, 'name': 'Casey Affleck'},
 {'movie_id': 429203, 'actor_id': 2047, 'order': 3, 'name': 'Danny Glover'},
 {'movie_id': 429203, 'actor_id': 2887, 'order': 4, 'name': 'Tom Waits'},
 {'movie_id': 429203, 'actor_id': 110742, 'order': 5, 'name': 'Tika Sumpter'},
 {'movie_id': 429203,
  'actor_id': 2200140,
  'order': 6,
  'name': 'Ari Elizabeth Johnson'},
 {'movie_id': 429203,
  'actor_id': 2200141,
  'order': 7,
  'name': 'Teagan Johnson'},
 {'movie_id': 429203, 'actor_id': 1194944, 'order': 8, 'name': 'Gene Jones'},
 {'movie_id': 429203,
  'actor_id': 1117313,
  'order': 9,
  'name': 'John David Washington'},
 {'movie_id': 429203, 'actor_id': 71885, 'order': 10, 'name': 'Barlow Jacobs'},
 {'movie_id': 429203,
  'actor_id': 1285848,
  'order': 11,
  'name': 'Augustine Frizzell'},
 {'movi

In [6]:
import pandas as pd
import mysql.connector
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
c = cnx.cursor()

In [7]:
q = """CREATE TABLE movie_actors (
       id int(7),
       actor VARCHAR(255),
       PRIMARY KEY (id, actor)
     ) ENGINE=InnoDB"""
c.execute(q)
cnx.commit()

ProgrammingError: 1050 (42S01): Table 'movie_actors' already exists

In [156]:
def actor_movie_formatting(details):
    formatted_actor_movies = []
    for actor_movie in details:
        actor_movie_tuple = (actor_movie['movie_id'], actor_movie['name'])
        formatted_actor_movies.append(actor_movie_tuple)
    return formatted_actor_movies

In [157]:
def add_to_db_actor_movie(actor_movie_list):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO movie_actors (id, actor) VALUES (%s, %s)"""
    c.executemany(stmt, actor_movie_list)
    cnx.commit()
    c.close()
    cnx.close()

In [158]:
add_to_db_actor_movie(actor_movie_formatting(movie_actors))

In [12]:
from bs4 import BeautifulSoup
import config
import requests
import json

In [164]:
response = requests.get('https://ethnicelebs.com/charlie-sheen')
soup = BeautifulSoup(response.content, 'html.parser')

In [179]:
inner = soup.find('div', class_='entry-inner')

In [211]:
inner.select("strong")[0].text.replace('Ethnicity: ', '').split(',')

['Galician Spanish (paternal grandfather)',
 ' Irish',
 ' English',
 ' Scottish',
 ' possibly other']

In [6]:
import pandas as pd
import mysql.connector
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
c = cnx.cursor()
q = """SELECT actor FROM movie_actors WHERE id IN(SELECT id from details WHERE language='en' AND production_country='US' AND revenue > 0 AND budget >0);"""
c.execute(q)
list_of_actors = c.fetchall()

In [7]:
len(list_of_actors)

6137

In [8]:
new_list_of_actors = list(set(list_of_actors))

In [61]:
len(new_list_of_actors)

2661

In [238]:
missing_actors = []
actor_ethnicities = []

In [1]:
import pandas as pd
import mysql.connector
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
c = cnx.cursor()

In [2]:
q = """CREATE TABLE actors (
       name VARCHAR(255),
       ethnicity VARCHAR(255)
     ) ENGINE=InnoDB"""
# q = """DROP TABLE actors"""
c.execute(q)
cnx.commit()

In [ ]:
        actor_ethnicity_dict = {}
        soup = BeautifulSoup(response.content, 'html.parser')
        inner = soup.find('strong')
        ethnicities_list = inner.select("strong")[0].text.replace('Ethnicity: ', '').split(',')
        actor_ethnicity_dict[actor[0]] = ethnicities_list
        actor_ethnicities.append(actor_ethnicity_dict)
        print(actor_ethnicity_dict)
        print(n)

In [1]:
len(missing_actors)

NameError: name 'missing_actors' is not defined

In [22]:
missing_actors = []


In [70]:
actor_ethnicities = []
for n, actor in enumerate(new_list_of_actors[2300:2661]):
#     print('https://ethnicelebs.com/' + actor[0].lower().replace(' ', '-'))
    
    response = requests.get('https://ethnicelebs.com/' + actor[0].lower().replace(' ', '-'))
    if response.status_code != 200:
        missing_actors.append(actor[0].lower().replace(' ', '-'))
        print(actor[0].lower().replace(' ', '-'))
    else:
        actor_ethnicity_dict = {}
        soup = BeautifulSoup(response.content, 'html.parser')
        inner = soup.find('strong')
        inner = inner.text.replace('Ethnicity:\n*', '')
        inner = inner.replace('Ethnicity:', '')
        inner = inner.replace('\n*', ', ')
        print(inner)
#         inner = inner.replace('possibly other', '')
#         inner = inner.replace('possibly ', '')
        inner = inner.replace('or ', ',')
        inner = inner.replace(' (father)', '')
        inner = inner.replace(' (mother)', '')
        inner = inner.replace(' (paternal grandfather)', '')
        inner = inner.replace('paternal grandfather', '')
        inner = inner.replace('maternal grandfather', '')
        inner = inner.replace('maternal grandmother', '')
        inner = inner.replace('paternal grandmother', '')
        inner = inner.replace('mother – ', '')
        inner = inner.replace('father – ', '')
        inner = inner.replace('with smaller amounts of ', '')
        inner = inner.replace('some ', '')
        inner = inner.replace('as well as remote ', '')
        inner = inner.replace('along with remote ', '')
        inner = inner.replace(' remote ', '')
        inner = inner.replace('1/4th ', '')
        inner = inner.replace('1/8th ', '')
        inner = inner.replace('1/8 ', '')
        inner = inner.replace('1/512 ', 'distant ')
        inner = inner.replace('1/1024th ', 'distant ')
        inner = inner.replace('one eighth ', '')
        inner = inner.replace('(one eighth)', '')
        inner = inner.replace('1/16th ', '')
        inner = inner.replace('1/16 ', '')
        inner = inner.replace('as well as ', '')
        inner = inner.replace('1/32', '')
        inner = inner.replace('3/16ths', '')
        inner = inner.replace('1/4', '')
        inner = inner.replace('75% ', '')
        inner = inner.replace('25% ', '')
        inner = inner.replace('50% ', '')
        inner = inner.replace('87.5% ', '')
        inner = inner.replace('12.5% ', '')
        inner = inner.replace('1/1024 ', '')
        inner = inner.replace('1/64th', ',')
        inner = inner.replace('1/64', '')
        inner = inner.replace('37.5% ', '')
        inner = inner.replace('62.5% ', '')
        inner = inner.replace('including ', '')
        inner = inner.replace('one fourth ', '')
        inner = inner.replace('(one quarter) ', '')
        inner = inner.replace('(37.5%) ', '')
        inner = inner.replace('and ', ',')
        inner = inner.replace('mix of ', '')
        inner = inner.replace(u'\xa0',u'')
        inner = inner.replace('/', ', ')
        ethnicities_list = inner.split(',')

        actor_ethnicity_dict[actor[0]] = ethnicities_list
        actor_ethnicities.append(actor_ethnicity_dict)
        print(actor_ethnicity_dict)
        print(n)
print(actor_ethnicities)
print(missing_actors)


owen-asztalos
father – Eastern European Ashkenazi Jewish, mother – Norwegian, as well as English, Scots-Irish/Northern Irish, and German
{'Ansel Elgort': ['Eastern European Ashkenazi Jewish', ' Norwegian', ' English', ' Scots-Irish', ' Northern Irish', ' ', 'German']}
1
william-demeo
Argentinian (father), French, Puerto Rican (mother)
{'Steve Lemme': ['Argentinian', ' French', ' Puerto Rican']}
3
ramón-rodríguez
English (father), Ashkenazi Jewish (mother)
{'Bel Powley': ['English', ' Ashkenazi Jewish']}
5
chris-j.-murray
jaeden-martell
German (father), Scottish, small amount of Irish (mother)
{'Christopher Walken': ['German', ' Scottish', ' small amount of Irish']}
8
English, Irish, Scottish, German (father), Ashkenazi Jewish (mother)
{'Ashley Tisdale': ['English', ' Irish', ' Scottish', ' German', ' Ashkenazi Jewish']}
9
libby-villari
Māori, some Welsh (father), Danish, French, English (mother)
{'Rena Owen': ['Māori', ' Welsh', ' Danish', ' French', ' English']}
11
 Scottish
{'Scott S

Moroccan Berber (father), Spanish (mother)
{'Lubna Azabal': ['Moroccan Berber', ' Spanish']}
96
 Swiss-German, German, English, Dutch, Scottish, distant Irish, French, and Welsh
{'Taylor Lautner': [' Swiss-German', ' German', ' English', ' Dutch', ' Scottish', ' distant Irish', ' French', ' ', 'Welsh']}
97
michael-arden
christopher-eccleston
37.5% Bohemian Czech, 25% Swedish, 25% German, 12.5% English
{'Crispin Glover': ['Bohemian Czech', ' Swedish', ' German', ' English']}
100
nicolas-martinez
75% mix of German, Irish, Scottish, English, distant Welsh, remote Dutch, 25% mix of Ashkenazi Jewish and more distant Sephardi Jewish
{'Lily Rabe': ['German', ' Irish', ' Scottish', ' English', ' distant Welsh', 'Dutch', ' Ashkenazi Jewish ', 'more distant Sephardi Jewish']}
102
lauren-vélez
 English
{'Lesley Manville': [' English']}
104
 Irish
{'Pierce Brosnan': [' Irish']}
105
 English
{'Jim Broadbent': [' English']}
106
onata-aprile
c.j.-wilson
 English
{'Julie Andrews': [' English']}
109
 E

 Welsh
{'Henry Thomas': [' Welsh']}
203
Italian (father), English, Scottish, Irish, German, remote Dutch (mother)
{'Owen Vaccaro': ['Italian', ' English', ' Scottish', ' Irish', ' German', 'Dutch']}
204
marion-lambert
morfydd-clark
 Scottish, likely Irish
{'Peter Mullan': [' Scottish', ' likely Irish']}
207
 English, Scottish, Irish, distant German
{'Nicole Kidman': [' English', ' Scottish', ' Irish', ' distant German']}
208
 African-American
{'Donald Glover': [' African-American']}
209
ashley-grace
 Swedish
{'Dolph Lundgren': [' Swedish']}
211
eric-jacobson
 English, Scottish, smaller amounts of German, Dutch, French, Swedish, and Welsh
{'Anna Camp': [' English', ' Scottish', ' smaller amounts of German', ' Dutch', ' French', ' Swedish', ' ', 'Welsh']}
213
jennifer-prior
valeria-cotto
 German, Irish, English, Scottish
{'James Cromwell': [' German', ' Irish', ' English', ' Scottish']}
216
 German, as well as Irish, English, and Dutch
{'William Fichtner': [' German', ' Irish', ' English

 English, German, Scottish, some Irish, more distant Swiss-German, remote Cornish and Jersey [Channel Islander]
{'Israel Broussard': [' English', ' German', ' Scottish', ' Irish', ' more distant Swiss-German', 'Cornish ', 'Jersey [Channel Islander]']}
308
iris-bahr
 Syrian (great-grandfather), Italian, Irish, possibly other
{'Charlie Tahan': [' Syrian (great-grandfather)', ' Italian', ' Irish', ' possibly other']}
310
lorraine-nicholson
 Ghanaian
{'Abraham Attah': [' Ghanaian']}
312
 Scottish
{'James McAvoy': [' Scottish']}
313
 English, Swedish, German, Austrian
{'Elisabeth Moss': [' English', ' Swedish', ' German', ' Austrian']}
314
 Armenian, English, likely some Arab
{'Andy Serkis': [' Armenian', ' English', ' likely Arab']}
315
andrew-lees
 African-American
{'Angela Bassett': [' African-American']}
317
 Danish, some Swedish
{'Nikolaj Coster-Waldau': [' Danish', ' Swedish']}
318
blythe-danner
rachael-taylor
ionut-grama
 African-American
{'Viola Davis': [' African-American']}
322
75

In [71]:
add_to_db_actor_ethnicities(actor_formatting(actor_ethnicities))

In [16]:
def actor_formatting(ethnicities_list):
    formatted_actor_ethnicities = []
    for actor in ethnicities_list:
        for key, value in actor.items():
            for ethnicity in value:
                actor_tuple = (key, ethnicity)
                formatted_actor_ethnicities.append(actor_tuple)
    return formatted_actor_ethnicities

In [17]:
def add_to_db_actor_ethnicities(formatted_actor_ethnicities):
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
    c = cnx.cursor()    
    stmt = """INSERT INTO actors (name, ethnicity) VALUES (%s, %s)"""
    c.executemany(stmt, formatted_actor_ethnicities)
    cnx.commit()
    c.close()
    cnx.close()

In [9]:
import pandas as pd
import mysql.connector
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
c = cnx.cursor()
q = """SET SQL_SAFE_UPDATES = 0;"""
c.execute(q)
cnx.commit()

# c = cnx.cursor()
# q = """DELETE FROM actors WHERE ethnicity = ''""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = RTRIM(LTRIM(ethnicity));"""
# c.execute(q)
# cnx.commit()

# c = cnx.cursor()
# q = """DELETE FROM actors WHERE ethnicity LIKE '%possibly%'""";
# c.execute(q)
# cnx.commit()

# q = """DELETE FROM actors WHERE ethnicity LIKE '%distant%'""";
# c.execute(q)
# cnx.commit()

# q = """DELETE FROM actors WHERE ethnicity LIKE '%small%amount%'""";
# c.execute(q)
# cnx.commit()

# q = """DELETE FROM actors WHERE ethnicity LIKE '%other%'""";
# c.execute(q)
# cnx.commit()


# q = """DELETE FROM actors WHERE ethnicity LIKE '%unknown%'""";
# c.execute(q)
# cnx.commit()

# q = """DELETE FROM actors WHERE ethnicity LIKE '%uncertain%'""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'along with', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'and', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'between', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'convert to', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'Birth Name', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, ':', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'approximately', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'around', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'at least', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '1', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '1.5' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '1024th' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '128th' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '16ths' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '30%', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '31.' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '32' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '40.6' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '43.' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '512th' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '56.' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '59.3' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '6.' , '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, ')', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '[', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, ']', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '16th' , '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '18.75%' , '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '18.' , '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '2048' , 'distant')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '256th' , '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '256' , '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '28.1' , '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '(', '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '3.1', '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '62.5%', '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'about', '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '68.', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '68.', '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '75%', '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '8', '')""";
# c.execute(q)
# cnx.commit()
# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '81.', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'one quarter', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'degree of', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'one eighth', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'mostly', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'predominantly', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'Ethnicity', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'from the', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'region', '')""";
# c.execute(q)
# cnx.commit()


# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'about', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'almost', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'Ethnicity', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'less', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '37.5%', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'to 25%', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'half', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'one half', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'likely', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'less', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'mainly', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'maternal grand', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'one sixteenth', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'father', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'Piedmontese', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'great-great-grandfather', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'great-grandfather', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'three fourths', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'converted to', '')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '-', ' ')""";
# c.execute(q)
# cnx.commit()

q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '–', ' ')""";
c.execute(q)
cnx.commit()


# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '3.', ' ')""";
# c.execute(q)
# cnx.commit()


# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '.', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, '45%', ' ')""";
# c.execute(q)
# cnx.commit()


# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'from', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'three quarters', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'one fourth', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'roughly', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'biologically', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'with', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'who', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'may have', ' ')""";
# c.execute(q)
# cnx.commit()

# q = """UPDATE actors SET ethnicity = REPLACE(ethnicity, 'great great gr', ' ')""";
# c.execute(q)
# cnx.commit()

q = """UPDATE actors SET ethnicity = RTRIM(LTRIM(ethnicity));"""
c.execute(q)
cnx.commit()

# q = """DELETE FROM actors WHERE ethnicity LIKE '%distant%'""";
# c.execute(q)
# cnx.commit()

q = """DELETE FROM actors WHERE ethnicity = ''""";
c.execute(q)
cnx.commit()






















# q = """DELETE FROM actors WHERE ethnicity LIKE '%likely%'""";
# c.execute(q)
# cnx.commit()


c.close()


True

In [11]:
import pandas as pd
import mysql.connector
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = config.database)
c = cnx.cursor()

# q = """ALTER TABLE actors
# ADD COLUMN nonwhite VARCHAR(15) AFTER ethnicity;"""
# c.execute(q)
# cnx.commit()
# c.close()

q = """UPDATE actors 
        set nonwhite=case 
        when ethnicity LIKE '%Chilean%' then 'non-white'
        when ethnicity LIKE '%Colombian%' then 'non-white'
        when ethnicity LIKE '%Mexican%' then 'non-white'
        when ethnicity LIKE '%Moroccan%' then 'non-white'
        when ethnicity LIKE '%Peruvian%' then 'non-white'
        when ethnicity LIKE '%Venezuelan%' then 'non-white'
        when ethnicity LIKE '%African%' then 'non-white'
        when ethnicity LIKE '%Afrikan%' then 'non-white'
        when ethnicity LIKE '%Puerto Rican%' then 'non-white'
        
        
        when ethnicity LIKE '%Irish%' then 'white'
        when ethnicity LIKE '%English%' then 'white'
        when ethnicity LIKE '%German%' then 'white'
        when ethnicity LIKE '%Swedish%' then 'white'
        when ethnicity LIKE '%Scottish%' then 'white'
        when ethnicity LIKE '%French%' then 'white'
        when ethnicity LIKE '%Dutch%' then 'white'
        when ethnicity LIKE '%Jewish%' then 'white'
        when ethnicity LIKE '%Welsh%' then 'white'
        when ethnicity LIKE '%Austrian%' then 'white'
        when ethnicity LIKE '%Norwegian%' then 'white'
        when ethnicity LIKE '%Danish%' then 'white'
        when ethnicity LIKE '%Finnish%' then 'white'
        when ethnicity LIKE '%Italian%' then 'white'
        when ethnicity LIKE '%Canadian%' then 'white'
        when ethnicity LIKE '%Belgian%' then 'white'
        when ethnicity LIKE '%Flemish%' then 'white'
        when ethnicity LIKE '%Swiss%' then 'white'
        when ethnicity LIKE '%Hungarian%' then 'white'
        when ethnicity LIKE '%Greek%' then 'white'
        when ethnicity LIKE '%Breton%' then 'white'
        when ethnicity LIKE '%Sorbian%' then 'white'
        when ethnicity LIKE '%Russian%' then 'white'
        when ethnicity LIKE '%Slovak%' then 'white'
        when ethnicity LIKE '%Croatian%' then 'white'
        when ethnicity LIKE '%Slovenian%' then 'white'
        when ethnicity LIKE '%Sicilian%' then 'white'
        when ethnicity LIKE '%Canadian%' then 'white'
        when ethnicity LIKE '%Cornish%' then 'white'
        when ethnicity LIKE '%Channel Islander%' then 'white'
        when ethnicity LIKE '%Jersey%' then 'white'
        when ethnicity LIKE '%Guernsey%' then 'white'
        when ethnicity LIKE '%Canadian%' then 'white'
        when ethnicity LIKE '%Spanish%' then 'white'
        when ethnicity LIKE '%Czech%' then 'white'
        when ethnicity LIKE '%Lithuanian%' then 'white'
        when ethnicity LIKE '%Scandinavian%' then 'white'
        when ethnicity LIKE '%Serbian%' then 'white'
        when ethnicity LIKE '%Bohemian%' then 'white'
        when ethnicity LIKE '%British%' then 'white'
        when ethnicity LIKE '%Walloon%' then 'white'
        when ethnicity LIKE '%Polish%' then 'white'
        when ethnicity LIKE '%Judaism%' then 'white'
        when ethnicity LIKE '%Catalan%' then 'white'
        when ethnicity LIKE '%Ecuadorian%' then 'white'
        when ethnicity LIKE '%European%' then 'white'
        when ethnicity LIKE '%Extremaduran%' then 'white'
        when ethnicity LIKE '%Frisian%' then 'white'
        when ethnicity LIKE '%Galacian%' then 'white'
        when ethnicity LIKE '%Icelandic%' then 'white'
        when ethnicity LIKE '%Isle of Man%' then 'white'
        when ethnicity LIKE '%Latvian%' then 'white'
        when ethnicity LIKE '%Rusyn%' then 'white'
        when ethnicity LIKE '%Moravian%' then 'white'
        when ethnicity LIKE '%Romanian%' then 'white'
        when ethnicity LIKE '%Ruthenian%' then 'white'
        when ethnicity LIKE '%Ukranian%' then 'white'
        when ethnicity LIKE '%Ulster%' then 'white'
        when ethnicity LIKE '%Portuguese%' then 'white'
        when ethnicity LIKE '%Maltese%' then 'white'
        when ethnicity LIKE '%Greenlic%' then 'white'
        when ethnicity LIKE '%Maltese%' then 'white'
        else 'non-white'
        end;"""
c.execute(q)
cnx.commit()

cnx.close()